In [279]:
from adam.core.computations import KinDynComputations
from adam.geometry import utils
import numpy as np
import casadi as cs
import math
import gym_ignition_models
from adam.core.computationsWithHardwareParameters import  KinDynComputationsWithHardwareParameters
from adam.geometry import linkParametric
from urdfpy import xyz_rpy_to_matrix, matrix_to_xyz_rpy

In [280]:
def SX2DM(sx):
    return cs.DM(sx)

# Loading the model 

In [281]:

urdf_path ="/home/carlotta/iit_ws/ergocub-gazebo-simulations/models/stickBot/model.urdf"
urdf_path_old = gym_ignition_models.get_model_file("iCubGazeboV2_5")
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
link_name_list = ['l_lower_leg']
link_name_list_empty  = []
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
r_hip_yaw_char = linkParametric.JointCharacteristics(-0.055989)
link_name = 'l_lower_leg'
link_characteristics = {link_name: r_hip_r_char}
joint_characteristics = {'l_ankle_pitch': r_hip_yaw_char}
print(joint_characteristics)
root_link = 'root_link'
kinDyn =KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list,link_name_list, root_link, link_characteristics, joint_characteristics)

kinDynNoParametric = KinDynComputations(urdf_path,joints_name_list, root_link)


{'l_ankle_pitch': <adam.geometry.linkParametric.JointCharacteristics object at 0x7ff9842f0a30>}


Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor


### Variable definition

In [282]:
# Set joints and base informations
link_original = kinDyn.get_element_by_name(link_name, kinDyn.robot)
r_hip_r_char = linkParametric.LinkCharacteristics(0.058)
length = link_original.visuals[0].geometry.cylinder.length 
radius = link_original.visuals[0].geometry.cylinder.radius
volume = math.pi*radius**2*length
mass = link_original.inertial.mass
density_num = mass/volume

xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
density =density_num 
lenght_multiplier = 1
gravity =np.array([0, 0, -9.80665, 0, 0, 0])

### Mass and Centroidal Matrix 

In [283]:
[M, J_cmm] = kinDyn.crba()
M = kinDyn.mass_matrix_fun() 
J_cmm = kinDyn.centroidal_momentum_matrix_fun()

# print("M", M(H_b, s, density, lenght_multiplier))
# print("J_cmm",J_cmm(H_b, s, density, lenght_multiplier))

In [284]:
M_no_parametric = kinDynNoParametric.mass_matrix_fun()
M_no_parametric_value = M_no_parametric(H_b, s)
M_value = M(H_b, s, density, lenght_multiplier)
# print("M", M_no_parametric(H_b, s,density, lenght_multiplier))



In [285]:
error = 0 
for i in range(29):
    for j in range(29):
        err = float(M_no_parametric_value[i,j] - M_value[i,j])
        error += abs(err)
print('Total Error Mass Matrices', error)

Total Error Mass Matrices 4.922331531091728e-12


In [286]:
J_cmm_no_parameric = kinDynNoParametric.centroidal_momentum_matrix_fun()
J_cmm_no_parameric_value = J_cmm_no_parameric(H_b,s)
J_cmm_value = J_cmm(H_b,s,density, lenght_multiplier)
error = 0 
for i in range(6):
    for j in range (29): 
        err = float(J_cmm_no_parameric_value[i,j] - J_cmm_value[i,j])
        error += abs(err)
print("Total error centroidal momentum matrix", error)



Total error centroidal momentum matrix 3.217124947400215e-12


### Total mass

In [287]:
mass = kinDyn.get_total_mass()
print('mass:\n',mass(density, lenght_multiplier))

mass:
 62.5739


In [288]:
mass_no_param = kinDynNoParametric.get_total_mass()
mass_value = mass(density, lenght_multiplier)
error = float(mass_no_param-mass_value)
print("error mass", error)

error mass 0.0


### Forward Kinematics

In [289]:
H = kinDyn.forward_kinematics_fun('l_sole')
print('H left sole :\n', H(H_b, s, density, lenght_multiplier))

H left sole :
 @1=0, 
[[0.98593, -0.166111, 0.0186687, 0.296073], 
 [0.139883, 0.758754, -0.63618, -0.437346], 
 [0.0915119, 0.629841, 0.771315, -2.36157], 
 [@1, @1, @1, 1]]


In [290]:
H_no_param = kinDynNoParametric.forward_kinematics_fun('l_sole')
H_no_param_value = H_no_param(H_b, s)
H_value = H(H_b, s, density,lenght_multiplier)
error = 0 
for i in range(4):
    for j in range(4):
        err = float(H_value[i,j]- H_no_param_value[i,j])
        error+= abs(err)

print("error forward kinematic", error)

error forward kinematic 0.0


### Center of Mass

In [291]:
com = kinDyn.CoM_position_fun()
print('CoM:\n',com(H_b, s, density, lenght_multiplier))

CoM:
 [0.186594, -0.293268, -2.33607]


### Jacobian

In [292]:
CoM_no_param = kinDynNoParametric.CoM_position_fun()
CoM_no_param_value = CoM_no_param(H_b,s)
CoM_value = com(H_b, s, density, lenght_multiplier)
error = 0
for i in range(3):
    err = float(CoM_no_param_value[i] - CoM_value[i])
    error+= abs(err)

print("error CoM", error)

error CoM 0.00011573823124103044


In [293]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian left sole\n',J(H_b, s, density, lenght_multiplier))

Jacobian left sole
 @1=1, @2=0, 
[[@1, @2, @2, @2, -0.049519, 0.151276, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.0546285, -0.0785708, -0.0317752, 0.176242, -0.0739164, -0.0109301, @2, @2, @2, @2, @2, @2], 
 [@2, @1, @2, 0.049519, @2, 0.186591, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.0250189, -0.214544, 0.0280053, -0.246504, 0.0125149, 0.049926, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @1, -0.151276, -0.186591, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.234138, 0.00957791, -0.215236, -0.0132259, -0.0043544, 0.0414435, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.59922, -0.50332, 0.0929803, 0.0766768, -0.0766768, 0.98593, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.769188, 0.146308, 0.989009, 0.108003, -0.108003, 0.139883, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.222001, -0.851624, 0.114958, -0.991189, 0.99118

In [294]:
J_no_param = kinDynNoParametric.jacobian_fun('l_sole')
J_no_param_value = J_no_param(H_b, s)
J_value = J(H_b,s, density, lenght_multiplier)
error = 0 
for i in range(6): 
    for j in range(29): 
        err = float(J_no_param_value[i,j]- J_value[i,j])
        error+ - abs(err)

print("error Jacobian", error)

error Jacobian 0


### Relative Jacobian

In [295]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian left sole relative: ', J_r(s,density, lenght_multiplier))

Jacobian left sole relative:  
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.240085, 0.0390649, 0.209228, 0.0324827, 0.0120337, -0.0477241, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.214232, -0.0502837, 0.081064, 0.0336993, -0.0226524, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0281009, -0.069805, 0.0426098, -0.290472, 0.066021, 0.0392291, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.872644, -0.298829, 0.937069, -0.937069, -0.247139, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -0.790929, -0.349066, 0.349066, -0.67807, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.488357, 0.533978, 0.00737389, -0.00737389, -0.692202, 0, 0, 0, 0, 0, 0]]


In [296]:
J_r_no_param = kinDynNoParametric.relative_jacobian_fun('l_sole')
J_r_no_param_value = J_r_no_param(s)
J_value = J_r(s, density, lenght_multiplier)
error = 0 
for i in range(6): 
    for j in range(23): 
        err = float(J_value[i,j] - J_no_param_value[i,j])
        error += abs(err)

print("error relative jacobian", error)

error relative jacobian 25.860947061945392


In [297]:
tau = kinDyn.rnea()
print('tau: \n', tau(H_b, s, v_b, s_dot, gravity, density, lenght_multiplier))

tau: 
 [15.5089, -38.9359, 662.622, -0.29743, -6.91321, -8.24292, -45.8233, -11.0071, 7.87415, -0.397646, 3.0006, 0.558566, -0.126913, 0.912939, -3.55099, -0.516252, -0.634119, -0.849935, 13.0357, 17.341, 2.99977e-05, -1.11876, -0.438037, 10.9559, -0.149108, -26.8917, -1.48014, 3.34249, 15.4686]


In [298]:
tau_no_parametric = kinDynNoParametric.rnea()

tau_no_parametric_value = tau_no_parametric(H_b, s, v_b, s_dot, gravity)
tau_value = tau(H_b, s, v_b, s_dot, gravity, density,  lenght_multiplier)

error = 0 
for i in range(29):
    err = float (tau_value[i] - tau_no_parametric_value[i])
    error += abs(err)

print("tau error", error)

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [ ]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot, density, lenght_multiplier)))

In [ ]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot, density, lenght_multiplier)))

In [ ]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s, density, lenght_multiplier)))

### Cylinder Link 

In [ ]:
link_original = kinDyn.get_element_by_name('l_lower_leg', kinDyn.robot)
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
print(link_original.visuals[0].geometry.cylinder)
length = link_original.visuals[0].geometry.cylinder.length 
radius = link_original.visuals[0].geometry.cylinder.radius
volume = math.pi*radius**2*length
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('radius=', radius)
print('length=', length)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])
link_parametric =linkParametric.linkParametric('l_lower_leg', 1.0,density_num, kinDyn.robot, link_original, r_hip_r_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new[1])
print('length=', link_parametric.visual_data_new[0])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


### Cylinder Link Parametric

In [ ]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_parametric =linkParametric.linkParametric('l_lower_leg', l_m,rho, kinDyn.robot, link_original, r_hip_r_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new[1])
print('length=', link_parametric.visual_data_new[0])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

### Box Link 

In [ ]:
link_original = kinDyn.get_element_by_name('chest', kinDyn.robot)
chest_char = linkParametric.LinkCharacteristics(0.0, linkParametric.Side.DEPTH, False, False)

width = link_original.visuals[0].geometry.box.size[0]
depth = link_original.visuals[0].geometry.box.size[2]
height = link_original.visuals[0].geometry.box.size[1]

volume = width*depth*height
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('width=', width)
print('depth=', depth)
print('height=', height)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])

link_parametric =linkParametric.linkParametric('chest', 1.0,density_num, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('width=', link_parametric.visual_data_new[0])
print('depth=', link_parametric.visual_data_new[2])
print('height=', link_parametric.visual_data_new[1])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


### Box Link Parametric

In [ ]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_parametric =linkParametric.linkParametric('chest', l_m,rho, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('width=', link_parametric.visual_data_new[0])
print('depth=', link_parametric.visual_data_new[2])
print('height=', link_parametric.visual_data_new[1])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

### Sphere Link

In [ ]:
link_original = kinDyn.get_element_by_name('l_elbow_1', kinDyn.robot)

radius = link_original.visuals[0].geometry.sphere.radius

volume = 4*(math.pi*radius**3)/3
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('radius=', radius)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])

link_parametric =linkParametric.linkParametric('l_elbow_1', 1.0,density_num, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new)
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

### Sphere Link Parametric 

In [ ]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_original = kinDyn.get_element_by_name('l_elbow_1', kinDyn.robot)
link_parametric =linkParametric.linkParametric('l_elbow_1', l_m,rho, kinDyn.robot, link_original)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new)
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


In [ ]:
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
r_hip_yaw_char = linkParametric.JointCharacteristics(-0.055989)
joint_original = kinDyn.get_joint_by_name('l_ankle_pitch',kinDyn.robot)
link_original = kinDyn.get_element_by_name('l_lower_leg', kinDyn.robot)
link_parametric =linkParametric.linkParametric('l_lower_leg', l_m,rho, kinDyn.robot, link_original, r_hip_r_char)
joint_parametric = linkParametric.jointParametric('l_ankle_pitch', link_parametric, joint_original, r_hip_yaw_char)
print(matrix_to_xyz_rpy(joint_original.origin))
print(joint_parametric.origin)

In [ ]:
Sc = cs.SX.zeros(6)
Sc = np.random.rand(6,6)
print(Sc)
# print(cs.mtimes(Sc,Sc.T))
# print(Sc @ Sc.T)
print(Sc[3:,:3])

In [ ]:
I = linkParametric.I_parametric(); 
I.ixx = 1
I.iyy = 2
I.izz = 3

I_float = np.zeros([3,3])
I_float[0,0] = 1 
I_float[1,1] = 2 
I_float[2,2] = 3

a = utils.spatial_inertial_with_parameter(I,1,[0,0,0], [0,0,0])
b = utils.spatial_inertia(I_float, 1,[0,0,0], [0,0,0])

print(a)
print(b)